# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# importing csv file from part I
weather_data = pd.read_csv('output_data/WeatherPy.csv')
weather_data.dropna(inplace = True) 
weather_data

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# group lats and lngs for each city location
coordinates = weather_data[["Lat","Lng"]]

# store humidity levels and convert to float
humidity = weather_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#sorted weather data for cities with wind sped of greater than 20mph, cloudiness less than 50,
#max temperatur between 40 & 90. Plus remove any rows with null values
filterd_weather_data = weather_data.loc[(weather_data["Wind Speed"] > 20) & (weather_data["Cloudiness"] < 50)  \
                                        & (weather_data["Max Temp"] >= 40) & (weather_data["Max Temp"] <= 90)]\
                                        .dropna().reset_index(drop=True)

filterd_weather_data





### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Create dataframe and empty column
hotel_df = filterd_weather_data.loc[:,["City","Country","Lat","Lng"]]

# adding extra column to the dataframe
hotel_df["Hotel Name"] = ""

#google api url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a parameters dictionary
params = {"keyword": "hotel",
          "radius": 5000,
          "type": "hotel",
          "key": g_key }

# Loop through dataframe and add nearby hotel result

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #cities coordnates and name
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    #adding location parameter to params
    params["location"] = f"{lat},{lng}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")      
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig
